# I fucking hate `pandas` and `sklearn` for their messy and inconsistent interfaces
# for their opposition to any extensions of their libraries
# and all that crap.
# I can't even sleep because of how much I hate them.
# I was trying to write things according to their API
# implementing BaseEstimator and TransformerMixin
# and using pipes
# but that makes things even worser
# because of uglyness of this environment!

# I wanted to try pretty obvious tricks and models
# but this environment makes it impossible

In [1]:
import pandas as pd
import numpy as np
import dateutil.parser
import datetime
import sklearn.preprocessing
import collections
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2

%matplotlib inline

In [2]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')

In [3]:
X_train.dtypes

id                         int64
amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_ty

In [4]:
def date_to_ymd(x):
    res = (np.nan, np.nan, np.nan)
    if isinstance(x, str):
        try:
            x = dateutil.parser.parse(x)
        except:
            pass
    if isinstance(x, datetime.date):
            res = (x.year, x.month, x.day)
    return res

In [5]:
X_train.drop(['id', 'recorded_by', 'num_private'], axis=1, inplace=True)
X_test.drop(['id', 'recorded_by', 'num_private'], axis=1, inplace=True)

In [6]:
(X_train.loc[:,'date_recorded_year'],
 X_train.loc[:,'date_recorded_month'],
 X_train.loc[:,'date_recorded_day']) = zip(*X_train.loc[:,'date_recorded'].map(date_to_ymd))
(X_test.loc[:,'date_recorded_year'],
 X_test.loc[:,'date_recorded_month'],
 X_test.loc[:,'date_recorded_day']) = zip(*X_test.loc[:,'date_recorded'].map(date_to_ymd))

X_train.drop('date_recorded', axis=1, inplace=True)
X_test.drop('date_recorded', axis=1, inplace=True)

In [7]:
X_train.loc[:,'age'] = X_train.loc[:,'date_recorded_year'] - X_train.loc[:,'construction_year']
X_test.loc[:,'age'] = X_test.loc[:,'date_recorded_year'] - X_test.loc[:,'construction_year']

In [8]:
X_train.loc[:,'amount_per_man'] = (X_train.loc[:,'amount_tsh']/X_train.loc[:,'population']).fillna(0)
X_test.loc[:,'amount_per_man'] = (X_test.loc[:,'amount_tsh']/X_test.loc[:,'population']).fillna(0)

In [9]:
X_train.loc[X_train.loc[:,'permit'] == True, 'permit'] = 1
X_train.loc[X_train.loc[:,'permit'] == False, 'permit'] = 0
X_train.loc[:,'permit'].fillna(0.5, inplace=True)
X_test.loc[X_test.loc[:,'permit'] == True, 'permit'] = 1
X_test.loc[X_test.loc[:,'permit'] == False, 'permit'] = 0
X_test.loc[:,'permit'].fillna(0.5, inplace=True)

In [10]:
X_train.loc[X_train.loc[:,'public_meeting'] == True, 'public_meeting'] = 1
X_train.loc[X_train.loc[:,'public_meeting'] == False, 'public_meeting'] = 0
X_train.loc[:,'public_meeting'].fillna(0.5, inplace=True)
X_test.loc[X_test.loc[:,'public_meeting'] == True, 'public_meeting'] = 1
X_test.loc[X_test.loc[:,'public_meeting'] == False, 'public_meeting'] = 0
X_test.loc[:,'public_meeting'].fillna(0.5, inplace=True)

In [11]:
X_train.loc[:,'wpt_name'] = X_train.loc[:,'wpt_name'].notnull() * 1
X_test.loc[:,'wpt_name'] = X_test.loc[:,'wpt_name'].notnull() * 1

In [12]:
X_train.loc[:,'source'] = (
    X_train.loc[:,'source'] + ' '
    + X_train.loc[:,'source_type'] + ' '
    + X_train.loc[:,'source_class']
)
X_test.loc[:,'source'] = (
    X_test.loc[:,'source'] + ' '
    + X_test.loc[:,'source_type'] + ' '
    + X_test.loc[:,'source_class']
)

In [13]:
X_train.loc[:,'quality'] = (
    X_train.loc[:,'water_quality'] + ' '
    + X_train.loc[:,'quality_group']
)
X_test.loc[:,'quality'] = (
    X_test.loc[:,'water_quality'] + ' '
    + X_test.loc[:,'quality_group']
)

In [14]:
X_train.loc[:,'waterpoint_type'] = (
    X_train.loc[:,'waterpoint_type'] + ' '
    + X_train.loc[:,'waterpoint_type_group']
)
X_test.loc[:,'waterpoint_type'] = (
    X_test.loc[:,'waterpoint_type'] + ' '
    + X_test.loc[:,'waterpoint_type_group']
)

In [15]:
X_train.loc[:,'quantity'] = (
    X_train.loc[:,'quantity'] + ' '
    + X_train.loc[:,'quantity_group']
)
X_test.loc[:,'quantity'] = (
    X_test.loc[:,'quantity'] + ' '
    + X_test.loc[:,'quantity_group']
)

In [16]:
X_train.loc[:,'management'] = (
    X_train.loc[:,'management'] + ' '
    + X_train.loc[:,'management_group']
)
X_test.loc[:,'management'] = (
    X_test.loc[:,'management'] + ' '
    + X_test.loc[:,'management_group']
)

In [17]:
X_train.loc[:,'scheme'] = (
    X_train.loc[:,'scheme_name'] + ' '
    + X_train.loc[:,'scheme_management']
)
X_test.loc[:,'scheme'] = (
    X_test.loc[:,'scheme_name'] + ' '
    + X_test.loc[:,'scheme_management']
)

In [18]:
X_train.loc[:,'payment'] = (
    X_train.loc[:,'payment'] + ' '
    + X_train.loc[:,'payment_type']
)
X_test.loc[:,'payment'] = (
    X_test.loc[:,'payment'] + ' '
    + X_test.loc[:,'payment_type']
)

In [19]:
X_train.loc[:,'extraction_type'] = (
    X_train.loc[:,'extraction_type'] + ' '
    + X_train.loc[:,'extraction_type_group'] + ' '
    + X_train.loc[:,'extraction_type_class']
)
X_test.loc[:,'extraction_type'] = (
    X_test.loc[:,'extraction_type'] + ' '
    + X_test.loc[:,'extraction_type_group'] + ' '
    + X_test.loc[:,'extraction_type_class']
)

In [20]:
drop_cats = ['source_type', 'source_class',
              'water_quality', 'quality_group',
              'quantity_group', 'waterpoint_type_group',
              'payment_type', 'extraction_type_group',
              'extraction_type_class', 'management_group',
             'scheme_name', 'scheme_management',
              ]
X_train.drop(drop_cats, axis=1, inplace=True, errors='ignore')
X_test.drop(drop_cats, axis=1, inplace=True, errors='ignore')

In [21]:
X_train.iloc[0,:]

amount_tsh                                              6000
funder                                                 Roman
gps_height                                              1390
installer                                              Roman
longitude                                            34.9381
latitude                                            -9.85632
wpt_name                                                   1
basin                                             Lake Nyasa
subvillage                                          Mnyusi B
region                                                Iringa
region_code                                               11
district_code                                              5
lga                                                   Ludewa
ward                                                Mundindi
population                                               109
public_meeting                                             1
permit                  

In [22]:
text_features = ['funder', 'installer', 'basin',
                'subvillage', 'region', 'region_code', 'district_code',
                'lga', 'ward', 
                 'quality',
                 #'water_quality', 'quality_group',
                 'scheme', #'scheme_management', 'scheme_name',
                'extraction_type', #'extraction_type_group',
                'management',# 'management_group', 
                 'payment',# 'payment_type',
                'quantity',# 'quantity_group',
                'source',# 'source_type', 'source_class',
                 'waterpoint_type', #'waterpoint_type_group',
                #'date_recorded_year', 'construction_year',
               ]
text_features_vectorizers = dict()
text_feats_train = []
text_feats_test = []

In [23]:
y_enc1 = sklearn.preprocessing.LabelEncoder().fit(y_train.iloc[:,1])
ytr = y_enc1.transform(y_train.iloc[:,1])

In [24]:
for cat in text_features:
    if X_train.loc[:,cat].dtype == object:
        X_train.loc[:,cat].fillna('unknown', inplace=True)
        X_train.loc[X_train.loc[:,cat].str.contains('other') == True, cat] = 'unknown'
        X_test.loc[:,cat].fillna('unknown', inplace=True)
        X_test.loc[X_test.loc[:,cat].str.contains('other') == True, cat] = 'unknown'
    else:
        print(cat)
        X_train.loc[:,cat].fillna(0, inplace=True)
        X_test.loc[:,cat].fillna(0, inplace=True)
        X_train.loc[:,cat] = X_train.loc[:,cat].astype(str)
        X_test.loc[:,cat] = X_test.loc[:,cat].astype(str)
    v = TfidfVectorizer(ngram_range=(1,2))
    v.fit(X_test.loc[:,cat])
    tr = v.transform(X_train.loc[:,cat])
    te = v.transform(X_test.loc[:,cat])
    kbest = SelectKBest(chi2, min(10, tr.shape[1])).fit(tr, ytr)
    tr = kbest.transform(tr)
    te = kbest.transform(te)
    tr = pd.DataFrame(tr.toarray()).add_prefix(cat + '_')
    te = pd.DataFrame(te.toarray()).add_prefix(cat + '_')
    text_feats_train.append(tr)
    text_feats_test.append(te)


X_train = pd.concat([X_train.drop(text_features, axis=1, errors='ignore')] + text_feats_train,
                    axis=1)
X_test = pd.concat([X_test.drop(text_features, axis=1, errors='ignore')] + text_feats_test,
                    axis=1)


region_code
district_code


In [25]:
display(X_train.head())

,amount_tsh,gps_height,longitude,latitude,wpt_name,population,public_meeting,permit,construction_year,date_recorded_year,...,waterpoint_type_0,waterpoint_type_1,waterpoint_type_2,waterpoint_type_3,waterpoint_type_4,waterpoint_type_5,waterpoint_type_6,waterpoint_type_7,waterpoint_type_8,waterpoint_type_9
0,6000.0,1390,34.938093,-9.856322,1,109,1.0,0.0,1999,2011,...,0.549138,0.549138,0.0,0.000000,0.000000,0.0,0.549138,0.308775,0.000000,0.0
1,0.0,1399,34.698766,-2.147466,1,280,0.5,1.0,2010,2013,...,0.549138,0.549138,0.0,0.000000,0.000000,0.0,0.549138,0.308775,0.000000,0.0
2,25.0,686,37.460664,-3.821329,1,250,1.0,1.0,2009,2013,...,0.395413,0.395413,0.0,0.420692,0.420692,0.0,0.395413,0.000000,0.420692,0.0
3,0.0,263,38.486161,-11.155298,1,58,1.0,1.0,1986,2013,...,0.395413,0.395413,0.0,0.420692,0.420692,0.0,0.395413,0.000000,0.420692,0.0
4,0.0,0,31.130847,-1.825359,1,0,1.0,1.0,0,2011,...,0.549138,0.549138,0.0,0.000000,0.000000,0.0,0.549138,0.308775,0.000000,0.0


In [26]:
[(c, X_train[c].dtype) for c in X_train.columns if not np.issubdtype(X_train[c].dtype, np.number)]

[]

In [27]:
def split_train_cv_test(X, y, proportions=(.75, .25/2, .25/2)):
    # expecting $y$ to be numpy array
    outs = np.unique(y)
    proportions = np.array(proportions)
    classes = [np.where(y == i)[0] for i in outs]
    xparts = [[] for _ in proportions]
    yparts = [[] for _ in proportions]
    for cidx in classes:
        cidx = sklearn.utils.shuffle(cidx)
        cprops = cidx.size * proportions
        cprops = cprops.astype(int)
        cprops[-1] = cidx.size - cprops[:-1].sum()
        # print(cidx)
        cx = X.iloc[cidx,:]
        cy = y[cidx]
        for xpart, ypart, sz in zip(xparts, yparts, cprops):
            xpart.append(cx.iloc[:sz,:])
            ypart.append(cy[:sz])
            cx, cy = cx.iloc[sz:,:], cy[sz:]
    xparts = [pd.concat(xpart) for xpart in xparts]
    yparts = [np.concatenate(ypart) for ypart in yparts]
    parts = xparts + yparts
    return parts

X_train.columns

Index(['amount_tsh', 'gps_height', 'longitude', 'latitude', 'wpt_name',
       'population', 'public_meeting', 'permit', 'construction_year',
       'date_recorded_year',
       ...
       'waterpoint_type_0', 'waterpoint_type_1', 'waterpoint_type_2',
       'waterpoint_type_3', 'waterpoint_type_4', 'waterpoint_type_5',
       'waterpoint_type_6', 'waterpoint_type_7', 'waterpoint_type_8',
       'waterpoint_type_9'],
      dtype='object', length=184)

In [28]:
def divide_by_tmax(f):
    X_train.loc[:,f] /= X_test.loc[:,f].max()
    X_test.loc[:,f] /= X_test.loc[:,f].max()
def extract_tmin(f):
    X_train.loc[:,f] -= X_test.loc[:,f].min()
    X_test.loc[:,f] -= X_test.loc[:,f].min()
    
divide_by_tmax('amount_tsh')
divide_by_tmax('population')
divide_by_tmax('age')
divide_by_tmax('amount_per_man')

extract_tmin('construction_year')
divide_by_tmax('construction_year')

X_train.drop(['date_recorded_year'], axis=1, inplace=True)
X_test.drop(['date_recorded_year'], axis=1, inplace=True)

In [29]:
xtr = X_train # ref
xte = X_test # ref
xtr, xcv, xva, ytr, ycv, yva = (
    split_train_cv_test(xtr, ytr, proportions=(.7, .15, .15))
)
y_bin = sklearn.preprocessing.LabelBinarizer().fit(ytr)
display(
    np.where(ytr != 0)[0][:40],
    np.where(ycv != 0)[0][:40],
    np.where(yva != 0)[0][:40],)
ytr = y_bin.transform(ytr)
ycv = y_bin.transform(ycv)
yva = y_bin.transform(yva)

array([22581, 22582, 22583, 22584, 22585, 22586, 22587, 22588, 22589,
       22590, 22591, 22592, 22593, 22594, 22595, 22596, 22597, 22598,
       22599, 22600, 22601, 22602, 22603, 22604, 22605, 22606, 22607,
       22608, 22609, 22610, 22611, 22612, 22613, 22614, 22615, 22616,
       22617, 22618, 22619, 22620])

array([4838, 4839, 4840, 4841, 4842, 4843, 4844, 4845, 4846, 4847, 4848,
       4849, 4850, 4851, 4852, 4853, 4854, 4855, 4856, 4857, 4858, 4859,
       4860, 4861, 4862, 4863, 4864, 4865, 4866, 4867, 4868, 4869, 4870,
       4871, 4872, 4873, 4874, 4875, 4876, 4877])

array([4840, 4841, 4842, 4843, 4844, 4845, 4846, 4847, 4848, 4849, 4850,
       4851, 4852, 4853, 4854, 4855, 4856, 4857, 4858, 4859, 4860, 4861,
       4862, 4863, 4864, 4865, 4866, 4867, 4868, 4869, 4870, 4871, 4872,
       4873, 4874, 4875, 4876, 4877, 4878, 4879])

In [30]:
def y_to_df(y, ids):
    y = y_bin.inverse_transform(y)
    y = y_enc1.inverse_transform(y)
    df = pd.DataFrame({'status_group': y}, index=ids)
    df.index.name = 'id'
    return df

In [31]:
xtr.columns.tolist()

['amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'wpt_name',
 'population',
 'public_meeting',
 'permit',
 'construction_year',
 'date_recorded_month',
 'date_recorded_day',
 'age',
 'amount_per_man',
 'funder_0',
 'funder_1',
 'funder_2',
 'funder_3',
 'funder_4',
 'funder_5',
 'funder_6',
 'funder_7',
 'funder_8',
 'funder_9',
 'installer_0',
 'installer_1',
 'installer_2',
 'installer_3',
 'installer_4',
 'installer_5',
 'installer_6',
 'installer_7',
 'installer_8',
 'installer_9',
 'basin_0',
 'basin_1',
 'basin_2',
 'basin_3',
 'basin_4',
 'basin_5',
 'basin_6',
 'basin_7',
 'basin_8',
 'basin_9',
 'subvillage_0',
 'subvillage_1',
 'subvillage_2',
 'subvillage_3',
 'subvillage_4',
 'subvillage_5',
 'subvillage_6',
 'subvillage_7',
 'subvillage_8',
 'subvillage_9',
 'region_0',
 'region_1',
 'region_2',
 'region_3',
 'region_4',
 'region_5',
 'region_6',
 'region_7',
 'region_8',
 'region_9',
 'region_code_0',
 'region_code_1',
 'region_code_2',
 'region_code_3',
 'region

In [32]:
X_train.to_csv('X_train_prepr.csv')
X_test.to_csv('X_test_prepr.csv')
pd.DataFrame(ytr, columns=y_enc1.classes_).to_csv('y_train_prepr2.csv')
#pd.Series(ytr).to_csv('y_train_prepr.csv')
pd.Series(y_enc1.classes_).to_csv('y_classes.csv')

In [33]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import keras.utils

Using TensorFlow backend.


In [34]:

clf = Sequential([
    Dense(2*xtr.shape[1], input_shape=(xtr.shape[1],), activation='softmax'),
    Dropout(.2),
    Dense(2*xtr.shape[1], activation='softmax'),
    Dense(2*xtr.shape[1], activation='softmax'),
    Dense(2*xtr.shape[1], activation='softmax'),
    Dense(3, activation='softmax'),
])

In [35]:
clf.compile(optimizer='rmsprop',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

In [39]:
np.isinf(xtr).sum().sum()

0

In [40]:
clf.fit(xtr.values, ytr,
      batch_size=32,
      epochs=1000)

Epoch 1/1000
41578/41578 [==============================] - 13s - loss: nan - acc: 0.5428    
Epoch 2/1000
41578/41578 [==============================] - 14s - loss: nan - acc: 0.5431    
Epoch 3/1000
41578/41578 [==============================] - 14s - loss: nan - acc: 0.5431    
Epoch 4/1000
41578/41578 [==============================] - 14s - loss: nan - acc: 0.5431    
Epoch 5/1000
41578/41578 [==============================] - 12s - loss: nan - acc: 0.5431    
Epoch 6/1000
41578/41578 [==============================] - 11s - loss: nan - acc: 0.5431    
Epoch 7/1000
41578/41578 [==============================] - 12s - loss: nan - acc: 0.5431    
Epoch 8/1000
41578/41578 [==============================] - 12s - loss: nan - acc: 0.5431    
Epoch 9/1000
41578/41578 [==============================] - 13s - loss: nan - acc: 0.5431    
Epoch 10/1000
41578/41578 [==============================] - 12s - loss: nan - acc: 0.5431    
Epoch 11/1000
41578/41578 [==============================] 

KeyboardInterrupt: 

In [ ]:
clf.score(xcv, ycv)

In [ ]:
clf.score(xva, yva)